In [17]:
import pandas as pd
from sklearn.datasets import load_wine, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [18]:
# Cell 2 - Define Config class
class Config:
    def __init__(self, dropout=0.5, learning_rate=0.001, num_epochs=100, batch_size=32):
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size

# Cell 3 - Define FeatureNN class
class FeatureNN(nn.Module):
    def __init__(self, config, name, input_shape, num_units, feature_num):
        super(FeatureNN, self).__init__()
        self.config = config
        self.name = name
        self.input_shape = input_shape
        self.num_units = num_units
        self.feature_num = feature_num
        self.fc = nn.Linear(input_shape, num_units)

    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

In [19]:
# Cell 4 - Define NAM class
class NAM(nn.Module):
    def __init__(self, config, name, num_inputs: int, num_units: int) -> None:
        super(NAM, self).__init__()
        
        # Assign the config to the class attribute
        self.config = config
        self._num_inputs = num_inputs
        
        # Initialize layers
        self.dropout = nn.Dropout(p=self.config.dropout)
        self.feature_nns = nn.ModuleList([
            FeatureNN(config=config, name=f'FeatureNN_{i}', input_shape=1, num_units=num_units, feature_num=i)
            for i in range(num_inputs)
        ])
        self.output_layer = nn.Linear(sum([num_units for _ in range(num_inputs)]), 3)
        self._bias = torch.nn.Parameter(data=torch.zeros(1))

    def calc_outputs(self, inputs: torch.Tensor):
        return [self.feature_nns[i](inputs[:, i:i+1]) for i in range(self._num_inputs)]

    def forward(self, inputs: torch.Tensor):
        individual_outputs = self.calc_outputs(inputs)
        conc_out = torch.cat(individual_outputs, dim=-1)
        dropout_out = self.dropout(conc_out)
        out = self.output_layer(dropout_out)
        return out, dropout_out

    def print_model_equation(self, feature_names):
        equation_terms = []
        feature_contributions = {}
        for i, fnn in enumerate(self.feature_nns):
            coefficients = fnn.fc.weight.data.flatten().tolist()
            intercepts = fnn.fc.bias.data.tolist()
            term = " + ".join([f"({coeff:.3f} * x_{feature_names[i]} + {intercept:.3f})" for coeff, intercept in zip(coefficients, intercepts)])
            equation_terms.append(term)
            feature_contributions[feature_names[i]] = sum(abs(c) for c in coefficients)
        equation = " + ".join(equation_terms) + f" + bias ({self._bias.item():.3f})"
        interpretability = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
        return interpretability[0][0], interpretability[-1][0]

In [20]:
# Cell 5 - Training function
# def train(model, X_train, y_train, config):
#     criterion = torch.nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
#     for epoch in range(config.num_epochs):
#         model.train()
#         optimizer.zero_grad()
#         outputs, _ = model(X_train)
#         loss = criterion(outputs, y_train)
#         loss.backward()
#         optimizer.step()
#     return model

def train(model, X_train, y_train, config):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    for epoch in range(config.num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs, _ = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        # Print loss at each epoch
        print(f"Epoch [{epoch+1}/{config.num_epochs}], Loss: {loss.item():.4f}")
        
    return model



In [21]:
# Cell 6 - Evaluation function
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs, _ = model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    return accuracy

In [22]:

# Cell 7 - Load dataset function
def load_dataset(dataset_name: str):
    if dataset_name == 'wine':
        data = load_wine()
    elif dataset_name == 'iris':
        data = load_iris()
    else:
        raise ValueError(f"Dataset {dataset_name} is not supported.")
    
    df = pd.DataFrame(data.data, columns=data.feature_names)
    df['target'] = data.target
    return df

In [23]:

# Cell 8 - Analyze features function
def analyze_features(dataset_name, nam_model_class, config_class, num_clients=3):
    df = load_dataset(dataset_name)
    print(df)
    target_column = 'target'

    # Split dataset for federated learning simulation
    n_clients = num_clients
    client_data = [df.iloc[i * len(df) // n_clients: (i + 1) * len(df) // n_clients] for i in range(n_clients)]
    feature_columns = df.columns.drop(target_column)

    clients_high_contrib = {}
    clients_low_contrib = {}

    for i, client_df in enumerate(client_data):
        X = client_df[feature_columns].values
        y = client_df[target_column].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Normalize the data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        config = config_class()
        num_inputs = len(feature_columns)
        nam_model = nam_model_class(config=config, name=f"Client_{i}_Model", num_inputs=num_inputs, num_units=10)

        # Train the model
        trained_model = train(nam_model, X_train_tensor, y_train_tensor, config)

        # Evaluate the model
        accuracy = evaluate(trained_model, X_test_tensor, y_test_tensor)
        print(f"Client {i} Accuracy: {accuracy * 100:.2f}%")

        # Get high and low contributing features
        high_contrib, low_contrib = trained_model.print_model_equation(feature_columns)
        clients_high_contrib[i] = high_contrib
        clients_low_contrib[i] = low_contrib

    return clients_high_contrib, clients_low_contrib

In [25]:
# Cell 9 - Main execution
if __name__ == "__main__":
    high_contrib, low_contrib = analyze_features(dataset_name='wine', nam_model_class=NAM, config_class=Config)
    print("High contributing features:", high_contrib)
    print("Low contributing features:", low_contrib)

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  nonflavanoid_phenols  proanthocyan

In [27]:
# Import necessary libraries
import pandas as pd
from sklearn.datasets import load_wine, load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# Define Config class
class Config:
    def __init__(self, dropout=0.5, learning_rate=0.001, num_epochs=100, batch_size=32):
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size

# Define FeatureNN class
class FeatureNN(nn.Module):
    def __init__(self, config, name, input_shape, num_units, feature_num):
        super(FeatureNN, self).__init__()
        self.fc = nn.Linear(input_shape, num_units)

    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

# Define NAM class
class NAM(nn.Module):
    def __init__(self, config, name, num_inputs: int, num_units: int, n_classes: int) -> None:
        super(NAM, self).__init__()
        self.config = config
        self._num_inputs = num_inputs
        self.dropout = nn.Dropout(p=self.config.dropout)
        self.feature_nns = nn.ModuleList([
            FeatureNN(config=config, name=f'FeatureNN_{i}', input_shape=1, num_units=num_units, feature_num=i)
            for i in range(num_inputs)
        ])
        self.output_layer = nn.Linear(num_inputs * num_units, n_classes)
        self._bias = torch.nn.Parameter(data=torch.zeros(1))

    def calc_outputs(self, inputs: torch.Tensor):
        return [self.feature_nns[i](inputs[:, i:i+1]) for i in range(self._num_inputs)]

    def forward(self, inputs: torch.Tensor):
        individual_outputs = self.calc_outputs(inputs)
        conc_out = torch.cat(individual_outputs, dim=-1)
        dropout_out = self.dropout(conc_out)
        out = self.output_layer(dropout_out)
        return out, dropout_out

    def print_model_equation(self, feature_names):
        feature_contributions = {}
        for i, fnn in enumerate(self.feature_nns):
            coefficients = fnn.fc.weight.data.flatten().tolist()
            feature_contributions[feature_names[i]] = sum(abs(c) for c in coefficients)
        interpretability = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
        return interpretability[0][0], interpretability[-1][0]

# Training function
def train(model, X_train, y_train, config):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    for epoch in range(config.num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs, _ = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{config.num_epochs}], Loss: {loss.item():.4f}")
    return model

# Evaluation function
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs, _ = model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    return accuracy

# Load dataset function
def load_dataset(dataset_name: str):
    if dataset_name == 'wine':
        data = load_wine()
    elif dataset_name == 'iris':
        data = load_iris()
    elif dataset_name == 'breast_cancer':
        data = load_breast_cancer()
    else:
        raise ValueError(f"Dataset {dataset_name} is not supported.")
    df = pd.DataFrame(data.data, columns=data.feature_names)
    df['target'] = data.target
    return df

# Analyze features function
def analyze_features(dataset_name, nam_model_class, config_class, num_clients=3):
    df = load_dataset(dataset_name)
    target_column = 'target'
    n_classes = df[target_column].nunique()

    # Split dataset for federated learning simulation
    n_clients = num_clients
    client_data = [df.iloc[i * len(df) // n_clients: (i + 1) * len(df) // n_clients] for i in range(n_clients)]
    feature_columns = df.columns.drop(target_column)

    clients_high_contrib = {}
    clients_low_contrib = {}

    for i, client_df in enumerate(client_data):
        X = client_df[feature_columns].values
        y = client_df[target_column].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Normalize the data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        config = config_class()
        num_inputs = len(feature_columns)
        nam_model = nam_model_class(
            config=config,
            name=f"Client_{i}_Model",
            num_inputs=num_inputs,
            num_units=10,
            n_classes=n_classes
        )

        # Train the model
        trained_model = train(nam_model, X_train_tensor, y_train_tensor, config)

        # Evaluate the model
        accuracy = evaluate(trained_model, X_test_tensor, y_test_tensor)
        print(f"Client {i} Accuracy: {accuracy * 100:.2f}%")

        # Get high and low contributing features
        high_contrib, low_contrib = trained_model.print_model_equation(feature_columns)
        clients_high_contrib[i] = high_contrib
        clients_low_contrib[i] = low_contrib

    return clients_high_contrib, clients_low_contrib

# Main execution
if __name__ == "__main__":
    high_contrib, low_contrib = analyze_features(
        dataset_name='breast_cancer',
        nam_model_class=NAM,
        config_class=Config
    )
    print("High contributing features:", high_contrib)
    print("Low contributing features:", low_contrib)


Epoch [1/100], Loss: 0.8228
Epoch [2/100], Loss: 0.7644
Epoch [3/100], Loss: 0.6938
Epoch [4/100], Loss: 0.6770
Epoch [5/100], Loss: 0.6459
Epoch [6/100], Loss: 0.6324
Epoch [7/100], Loss: 0.5929
Epoch [8/100], Loss: 0.5480
Epoch [9/100], Loss: 0.5717
Epoch [10/100], Loss: 0.5514
Epoch [11/100], Loss: 0.5151
Epoch [12/100], Loss: 0.5040
Epoch [13/100], Loss: 0.4354
Epoch [14/100], Loss: 0.4758
Epoch [15/100], Loss: 0.4592
Epoch [16/100], Loss: 0.4136
Epoch [17/100], Loss: 0.4374
Epoch [18/100], Loss: 0.3987
Epoch [19/100], Loss: 0.4194
Epoch [20/100], Loss: 0.3847
Epoch [21/100], Loss: 0.3752
Epoch [22/100], Loss: 0.3656
Epoch [23/100], Loss: 0.3565
Epoch [24/100], Loss: 0.3502
Epoch [25/100], Loss: 0.3456
Epoch [26/100], Loss: 0.3353
Epoch [27/100], Loss: 0.3133
Epoch [28/100], Loss: 0.3236
Epoch [29/100], Loss: 0.3075
Epoch [30/100], Loss: 0.2850
Epoch [31/100], Loss: 0.2934
Epoch [32/100], Loss: 0.3103
Epoch [33/100], Loss: 0.2941
Epoch [34/100], Loss: 0.2803
Epoch [35/100], Loss: 0

# FOR UCI Wine dataset

In [3]:

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Subset, TensorDataset
# import numpy as np
# from sklearn import datasets
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from typing import Sequence, Tuple
# import pandas as pd
# from sklearn.datasets import fetch_openml
# from collections import OrderedDict, defaultdict
# import torch.nn.functional as F
# import os
# model_equations = []

# def fed_model(testimages):
#     wine = fetch_openml(name='wine-quality-red', version=1, as_frame=True, parser='liac-arff') 
#     X = wine.data
#     y = wine.target
#     y = wine.target.astype(int)
#     def adjust_target(value):
#         if value == 3:
#             return 0
#         elif value == 9:
#             return 6
#         else:
#             return value - 3
#     y = y.apply(adjust_target)
#     scaler = StandardScaler()
#     X = scaler.fit_transform(X)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    
#     y_train = y_train.astype(int)
#     y_train_np = y_train.to_numpy().astype(int)
#     y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test = y_test.astype(int)
#     y_test_np = y_test.to_numpy()
#     y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)
#     train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     n_clients = 3

#     indices = np.arange(len(train_dataset))
#     np.random.shuffle(indices)
#     split_indices = np.array_split(indices, n_clients)

#     client_loaders = []
#     batch_size = config.batch_size
#     for client_indices in split_indices:
#         client_subset = Subset(train_dataset, client_indices)
#         client_loader = DataLoader(client_subset, batch_size=batch_size, shuffle=True)
#         client_loaders.append(client_loader)

#     class SimpleNN(nn.Module):
#         def __init__(self):
#             super(SimpleNN, self).__init__()
#             self.fc1 = nn.Linear(11, 50)
#             self.fc2 = nn.Linear(50, 20)
#             self.fc3 = nn.Linear(20, 6)
#             self.relu = nn.ReLU()

#         def forward(self, x):
#             x = self.relu(self.fc1(x))
#             x = self.relu(self.fc2(x))
#             x = self.fc3(x)
#             return x

#     client_models = []
#     epochs = config.num_epochs
#     criterion = nn.CrossEntropyLoss()

#     for i, loader in enumerate(client_loaders):
#         model = SimpleNN()
#         optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#         for epoch in range(epochs):
#             running_loss = 0.0
#             for inputs, labels in loader:
#                 optimizer.zero_grad()
#                 outputs = model(inputs)
#                 loss = criterion(outputs, labels)
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item()
#         save_dir = '/Volumes/T7 Shield/Project/FedNAMS/saved_models'
#         os.makedirs(save_dir, exist_ok=True)
#         save_path = os.path.join(save_dir, f'client_{i+1}_model.pth')
#         torch.save(model.state_dict(), save_path)

#         # torch.save(model.state_dict(), f'client_{i+1}_model.pth')
#         client_models.append(model.state_dict())
#         sums = defaultdict(int)
#         count = len(client_models)
#         for od in client_models:
#             for key, value in od.items():
#                 sums[key] += value

#         averages = {key: value / count for key, value in sums.items()}
#         average_ordereddict = OrderedDict(averages)

#         model.load_state_dict(average_ordereddict)
#         testimages = torch.tensor(testimages, dtype=torch.float32)
#         y_test = model(testimages)
#         return y_test
# class Config:
#     def __init__(self, dropout=0.5, learning_rate=0.001, num_epochs=100, batch_size=32):
#         self.dropout = dropout
#         self.learning_rate = learning_rate
#         self.num_epochs = num_epochs
#         self.batch_size = batch_size

# class Model(nn.Module):
#     def __init__(self, config, name):
#         super(Model, self).__init__()
#         self.config = config
#         self.name = name
# class FeatureNN(nn.Module):
#     def __init__(self, config, name, input_shape, num_units, feature_num):
#         super(FeatureNN, self).__init__()
#         self.config = config
#         self.name = name
#         self.input_shape = input_shape
#         self.num_units = num_units
#         self.feature_num = feature_num
#         self.fc = nn.Linear(input_shape, num_units)

#     def forward(self, x):
#         x = self.fc(x)
#         x = F.relu(x)
#         return x

# class NAM(Model):
#     def __init__(self, config, name, *, num_inputs: int, num_units: int) -> None:
#         super(NAM, self).__init__(config, name)
#         self._num_inputs = num_inputs
#         self.dropout = nn.Dropout(p=self.config.dropout)

#         if isinstance(num_units, list):
#             assert len(num_units) == num_inputs
#             self._num_units = num_units
#         elif isinstance(num_units, int):
#             self._num_units = [num_units for _ in range(self._num_inputs)]

#         self.feature_nns = nn.ModuleList([
#             FeatureNN(config=config, name=f'FeatureNN_{i}', input_shape=1, num_units=self._num_units[i], feature_num=i)
#             for i in range(num_inputs)
#         ])

#         self.output_layer = nn.Linear(sum(self._num_units), 3)
#         self._bias = torch.nn.Parameter(data=torch.zeros(1))

#     def calc_outputs(self, inputs: torch.Tensor) -> Sequence[torch.Tensor]:
#         return [self.feature_nns[i](inputs[:, i:i+1]) for i in range(self._num_inputs)]

#     def forward(self, inputs: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
#         individual_outputs = self.calc_outputs(inputs)
#         conc_out = torch.cat(individual_outputs, dim=-1)
#         dropout_out = self.dropout(conc_out)
#         out = self.output_layer(dropout_out)
#         return out, dropout_out

#     def print_model_equation(self, feature_names):
#         equation_terms = []
#         feature_contributions = {}
#         print("feature_names")
#         print(feature_names)
#         for i, fnn in enumerate(self.feature_nns):
#             coefficients = fnn.fc.weight.data.flatten().tolist()
#             intercepts = fnn.fc.bias.data.tolist()
#             term = " + ".join([f"({coeff:.3f} * x_{feature_names[i]} + {intercept:.3f})" for coeff, intercept in zip(coefficients, intercepts)])
#             equation_terms.append(term)
#             feature_contributions[feature_names[i]] = sum(abs(c) for c in coefficients)
#         equation = " + ".join(equation_terms) + f" + bias ({self._bias.item():.3f})"
#         print(f"Model Equation: y = {equation}")
#         model_equations.append(equation)
#         interpretability = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
#         print("\nFeature Contributions:")
#         for feature, contribution in interpretability:
#             print(f"{feature}: {contribution:.3f}")

#         return interpretability[0][0],interpretability[-1][0]   # Return the feature with the highest contribution

# n_clients = 3
# data = fetch_openml(name='wine-quality-red', version=1, as_frame=True, parser='liac-arff')
# df = pd.DataFrame(data.data, columns=data.feature_names)
# df['target'] = data.target
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# client_data = [df.iloc[i * len(df) // n_clients: (i + 1) * len(df) // n_clients] for i in range(n_clients)]
# clients_features = [client.drop(columns=['target']) for client in client_data]
# clients_targets = [client['target'] for client in client_data]
# clients_targets = [target.astype(int) for target in clients_targets]
# clients_features = [client.drop(columns=['target']) for client in client_data]

# clients_features1 = {}
# clients_features2 = {}
# for i in range(n_clients):
#     feature_columns = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol'] 

#     df = clients_features[i].head()

#     def adjust_target(value):
#         return value - 3

#     target = clients_targets[i].head()
#     X = df[feature_columns].values
#     y = target
#     y = target.apply(adjust_target)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train = y_train.astype(int)
#     y_train_np = y_train.to_numpy().astype(int)
#     y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)

#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test = y_test.astype(int)
#     y_test_np = y_test.to_numpy()
#     y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)
#     config = Config(dropout=0.5, learning_rate=0.001, num_epochs=100, batch_size=32)
#     num_inputs = len(feature_columns)  # Number of features
#     num_units = 10  # Number of units in the hidden layer
#     nam_model = NAM(config=config, name='NAM_Model', num_inputs=num_inputs, num_units=num_units)

    
#     def train(model, X_train, y_train, config):
#         criterion = nn.CrossEntropyLoss()
#         optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
#         model.train()
#         for epoch in range(config.num_epochs):
#             outputs = fed_model(X_test_tensor)
#             optimizer.zero_grad()
#             y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
#             loss = criterion(outputs, y_test_tensor)
#             loss.backward()
#             optimizer.step()
#             if (epoch + 1) % 10 == 0:
#                 print(f'Epoch [{epoch + 1}/{config.num_epochs}], Loss: {loss.item():.4f}')
#         return model

#     def evaluate(model, X_test, y_test):
#         model.eval()
#         with torch.no_grad():
#             outputs, _ = model(X_test_tensor)
#             _, predicted = torch.max(outputs, 1)
#             accuracy = (predicted == y_test).sum().item() / y_test.size(0)
#             print(f'Accuracy: {accuracy * 100:.2f}%')

#     trained_model = train(nam_model, X_train_tensor, y_train_tensor, config)

#     evaluate(trained_model, X_test_tensor, y_test_tensor)

#     # Print the model equation and get the most contributing feature
#     a,b = trained_model.print_model_equation(feature_columns)
#     most_contributing_feature = a
#     clients_features1[i] = most_contributing_feature
#     least_contributing_feature = b
#     clients_features2[i] = least_contributing_feature
#     print(f"\nMost contributing feature for client's output {i+1}: {most_contributing_feature}")
#     print(f"Least contributing feature for client {i+1}: {least_contributing_feature}")


# # pip install openai==0.28

# # !pip install openai --upgrade


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import OrderedDict, defaultdict
import torch.nn.functional as F
from sklearn.metrics import classification_report, roc_auc_score
import itertools

# Model equations list
model_equations = []

# Loading UCI Heart Disease Dataset
def load_uci_wine_data():
    url = "winequality-red.csv"
    df = pd.read_csv(url, na_values="?")
    df = df.dropna()  # Drop rows with missing values
    df.apply(pd.to_numeric, errors='coerce')
    # Adjust the target to make it binary (0 for no heart disease, 1 for heart disease)
    df['target'] = df['quality'].apply(lambda x: 1 if x >= 6 else 0)
    df = df.drop(columns=['quality'])  # Drop original quality column
    return df

# Fetching the dataset
wine_data = load_uci_wine_data()

# Preprocessing the dataset
feature_columns = wine_data.columns[:-1] 
X = wine_data[feature_columns].values
y = wine_data['target'].values

# Scaling features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Split data into 3 clients
n_clients = 3
client_data = [X_train_tensor[i * len(X_train_tensor) // n_clients: (i + 1) * len(X_train_tensor) // n_clients] for i in range(n_clients)]
client_labels = [y_train_tensor[i * len(y_train_tensor) // n_clients: (i + 1) * len(y_train_tensor) // n_clients] for i in range(n_clients)]

# Configuration class
class Config:
    def __init__(self, dropout=0.5, learning_rate=0.001, num_epochs=100, batch_size=16, patience=15):
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.patience = patience

# Define FeatureNN and NAM models
class FeatureNN(nn.Module):
    def __init__(self, config, name, input_shape, num_units, feature_num):
        super(FeatureNN, self).__init__()
        self.config = config
        self.name = name
        self.input_shape = input_shape
        self.num_units = num_units
        self.feature_num = feature_num
        self.fc = nn.Linear(input_shape, num_units)

    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

class NAM(nn.Module):
    def __init__(self, config, name, *, num_inputs: int, num_units: int) -> None:
        super(NAM, self).__init__()
        self.config = config
        self.name = name
        self.num_inputs = num_inputs
        self.dropout = nn.Dropout(p=self.config.dropout)

        if isinstance(num_units, list):
            assert len(num_units) == num_inputs
            self._num_units = num_units
        elif isinstance(num_units, int):
            self._num_units = [num_units for _ in range(self.num_inputs)]

        self.feature_nns = nn.ModuleList([
            FeatureNN(config=config, name=f'FeatureNN_{i}', input_shape=1, num_units=self._num_units[i], feature_num=i)
            for i in range(num_inputs)
        ])
        self.output_layer = nn.Linear(sum(self._num_units), 2)  # 2 classes for binary classification
        self._bias = torch.nn.Parameter(data=torch.zeros(1))

    def calc_outputs(self, inputs: torch.Tensor):
        return [self.feature_nns[i](inputs[:, i:i+1]) for i in range(self.num_inputs)]

    def forward(self, inputs: torch.Tensor):
        individual_outputs = self.calc_outputs(inputs)
        conc_out = torch.cat(individual_outputs, dim=-1)
        dropout_out = self.dropout(conc_out)
        out = self.output_layer(dropout_out)
        return out, dropout_out

    def print_model_equation(self, feature_names):
        equation_terms = []
        feature_contributions = {}
        for i, fnn in enumerate(self.feature_nns):
            coefficients = fnn.fc.weight.data.flatten().tolist()
            intercepts = fnn.fc.bias.data.tolist()
            term = " + ".join([f"({coeff:.3f} * x_{feature_names[i]} + {intercept:.3f})" for coeff, intercept in zip(coefficients, intercepts)])
            equation_terms.append(term)
            feature_contributions[feature_names[i]] = sum(abs(c) for c in coefficients)
        equation = " + ".join(equation_terms) + f" + bias ({self._bias.item():.3f})"
        print(f"Model Equation: y = {equation}")
        model_equations.append(equation)
        interpretability = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
        print("\nFeature Contributions:")
        for feature, contribution in interpretability:
            print(f"{feature}: {contribution:.3f}")

        return interpretability[0][0], interpretability[-1][0]

# Custom weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Early stopping function
def early_stopping(val_loss, best_loss, stop_counter, patience):
    if val_loss < best_loss:
        best_loss = val_loss
        stop_counter = 0
    else:
        stop_counter += 1
    if stop_counter >= patience:
        print("Early stopping triggered!")
        return True, best_loss, stop_counter
    return False, best_loss, stop_counter

# Model training and evaluation functions with early stopping and scheduler
def train(model, train_loader, val_loader, config):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    model.apply(init_weights)  # Apply custom weight initialization
    model.train()

    best_loss = float('inf')
    stop_counter = 0
    for epoch in range(config.num_epochs):
        total_loss = 0
        model.train()
        for X_batch, y_batch in train_loader:
            outputs, _ = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation phase
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_val, y_val in val_loader:
                outputs, _ = model(X_val)
                loss = criterion(outputs, y_val)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        # Early stopping check
        stop, best_loss, stop_counter = early_stopping(val_loss, best_loss, stop_counter, config.patience)
        if stop:
            break
        
        scheduler.step()  # Update learning rate

        if (epoch + 1) % 10 == 0:
            avg_loss = total_loss / len(train_loader)
            print(f'Epoch [{epoch + 1}/{config.num_epochs}], Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}')
    return model

def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs, _ = model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).sum().item() / y_test.size(0)
        print(f'Accuracy: {accuracy * 100:.2f}%')
        # More detailed metrics
        report = classification_report(y_test, predicted)
        auc = roc_auc_score(y_test, predicted)
        print(f"Classification Report:\n{report}")
        print(f"ROC-AUC: {auc:.2f}")

# Hyperparameter tuning setup
dropouts = [0.3, 0.5]
learning_rates = [0.001, 0.0001]
num_units_list = [10]
batch_size = [16,32]


best_accuracy = 0.0
best_params = None

clients_features1 = {}
clients_features2 = {}

for dropout, lr, num_units, bs in itertools.product(dropouts, learning_rates, num_units_list, batch_size):
    print(f"Tuning with dropout={dropout}, learning_rate={lr}, num_units={num_units}, batch_size={bs}")
    for i in range(n_clients):
        config = Config(dropout=dropout, learning_rate=lr, num_epochs=100, batch_size=bs, patience=15)
        num_inputs = len(feature_columns)  # Number of features
        nam_model = NAM(config=config, name=f'NAM_Model_Client_{i+1}', num_inputs=num_inputs, num_units=num_units)
        
        # Create DataLoader for each client
        train_dataset = TensorDataset(client_data[i], client_labels[i])
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=config.batch_size)

        # Train the model with early stopping and scheduler
        trained_model = train(nam_model, train_loader, val_loader, config)

        # Evaluate the model
        evaluate(trained_model, X_test_tensor, y_test_tensor)
        accuracy = (trained_model(X_test_tensor)[0].argmax(dim=1) == y_test_tensor).float().mean().item()
        
        # Track best hyperparameters based on accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (dropout, lr, num_units, bs)

        # Print the model equation and get the most and least contributing features
        most_contributing_feature, least_contributing_feature = trained_model.print_model_equation(feature_columns)
        clients_features1[i] = most_contributing_feature
        clients_features2[i] = least_contributing_feature
        print(f"\nMost contributing feature for client {i+1}: {most_contributing_feature}")
        print(f"Least contributing feature for client {i+1}: {least_contributing_feature}")

# Print the best hyperparameters
print(f"\nBest Hyperparameters: Dropout: {best_params[0]}, Learning Rate: {best_params[1]}, Num Units: {best_params[2]},batch size: {best_params[3]}")
print(f"Best Validation Accuracy: {best_accuracy * 100:.2f}%")


Tuning with dropout=0.3, learning_rate=0.001, num_units=10, batch_size=16
Epoch [10/100], Loss: 0.5045, Val Loss: 0.5474
Epoch [20/100], Loss: 0.4834, Val Loss: 0.5377
Epoch [30/100], Loss: 0.5120, Val Loss: 0.5395
Early stopping triggered!
Accuracy: 73.33%
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.79      0.73       213
           1       0.81      0.69      0.74       267

    accuracy                           0.73       480
   macro avg       0.74      0.74      0.73       480
weighted avg       0.74      0.73      0.73       480

ROC-AUC: 0.74
Model Equation: y = (-0.405 * x_fixed acidity + -0.006) + (-0.325 * x_fixed acidity + 0.004) + (-0.076 * x_fixed acidity + -0.003) + (0.610 * x_fixed acidity + 0.070) + (0.108 * x_fixed acidity + 0.025) + (-0.102 * x_fixed acidity + 0.019) + (-0.656 * x_fixed acidity + 0.010) + (0.626 * x_fixed acidity + -0.041) + (0.666 * x_fixed acidity + 0.001) + (-0.631 * x_fixed acidity 